In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [2]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [3]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {char: i for i, char in enumerate(world_set)}

In [4]:
print(vocab)

{"'": 0, 'n': 1, 'y': 2, 'r': 3, 't': 4, 'c': 5, 'k': 6, 'f': 7, '.': 8, 's': 9, 'h': 10, 'a': 11, 'p': 12, 'i': 13, 'w': 14, 'e': 15, ' ': 16, 'd': 17, 'o': 18, 'b': 19, ',': 20, 'u': 21, 'B': 22, 'l': 23, 'g': 24, 'm': 25}


In [6]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [7]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 15  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

# sequence_length에 따라 다른 결과?
# 너무 길면 under fit, 너무 짧으면 over fit 가능성

In [8]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, 
##         y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i + sequence_length]
  y_str = sentence[i+1 : i+1 + sequence_length]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick walls are -> rick walls are 
1 rick walls are  -> ick walls are t
2 ick walls are t -> ck walls are th
3 ck walls are th -> k walls are the
4 k walls are the ->  walls are ther
5  walls are ther -> walls are there
6 walls are there -> alls are there 
7 alls are there  -> lls are there f
8 lls are there f -> ls are there fo
9 ls are there fo -> s are there for
10 s are there for ->  are there for 
11  are there for  -> are there for a
12 are there for a -> re there for a 
13 re there for a  -> e there for a r
14 e there for a r ->  there for a re
15  there for a re -> there for a rea
16 there for a rea -> here for a reas
17 here for a reas -> ere for a reaso
18 ere for a reaso -> re for a reason
19 re for a reason -> e for a reason 
20 e for a reason  ->  for a reason a
21  for a reason a -> for a reason an
22 for a reason an -> or a reason and
23 or a reason and -> r a reason and 
24 r a reason and  ->  a reason and y
25  a reason and y -> a reason and yo
26 a reason and yo -> 

In [9]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[22, 3, 13, 5, 6, 16, 14, 11, 23, 23, 9, 16, 11, 3, 15]
[3, 13, 5, 6, 16, 14, 11, 23, 23, 9, 16, 11, 3, 15, 16]


In [15]:
x_data

[[22, 3, 13, 5, 6, 16, 14, 11, 23, 23, 9, 16, 11, 3, 15],
 [3, 13, 5, 6, 16, 14, 11, 23, 23, 9, 16, 11, 3, 15, 16],
 [13, 5, 6, 16, 14, 11, 23, 23, 9, 16, 11, 3, 15, 16, 4],
 [5, 6, 16, 14, 11, 23, 23, 9, 16, 11, 3, 15, 16, 4, 10],
 [6, 16, 14, 11, 23, 23, 9, 16, 11, 3, 15, 16, 4, 10, 15],
 [16, 14, 11, 23, 23, 9, 16, 11, 3, 15, 16, 4, 10, 15, 3],
 [14, 11, 23, 23, 9, 16, 11, 3, 15, 16, 4, 10, 15, 3, 15],
 [11, 23, 23, 9, 16, 11, 3, 15, 16, 4, 10, 15, 3, 15, 16],
 [23, 23, 9, 16, 11, 3, 15, 16, 4, 10, 15, 3, 15, 16, 7],
 [23, 9, 16, 11, 3, 15, 16, 4, 10, 15, 3, 15, 16, 7, 18],
 [9, 16, 11, 3, 15, 16, 4, 10, 15, 3, 15, 16, 7, 18, 3],
 [16, 11, 3, 15, 16, 4, 10, 15, 3, 15, 16, 7, 18, 3, 16],
 [11, 3, 15, 16, 4, 10, 15, 3, 15, 16, 7, 18, 3, 16, 11],
 [3, 15, 16, 4, 10, 15, 3, 15, 16, 7, 18, 3, 16, 11, 16],
 [15, 16, 4, 10, 15, 3, 15, 16, 7, 18, 3, 16, 11, 16, 3],
 [16, 4, 10, 15, 3, 15, 16, 7, 18, 3, 16, 11, 16, 3, 15],
 [4, 10, 15, 3, 15, 16, 7, 18, 3, 16, 11, 16, 3, 15, 11],
 [10, 15, 3

In [16]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
# np.eye : 주 대각선이 모두 1이고 나머지는 0인 2차원 배열을 생성

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까??
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
# torch.FloatTensor : 32비트 부동 소수점 숫자를 저장
# torch.LongTensor : 64비트 정수(부호 있는 정수)를 저장

# 왜 X data 원핫인코딩은 0. 1. 이런 형태?

In [17]:
print(type(x_one_hot))
print(type(y_data))

<class 'list'>
<class 'list'>


In [18]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([183, 15, 26])
레이블의 크기 : torch.Size([183, 15])


In [19]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [20]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([ 3, 13,  5,  6, 16, 14, 11, 23, 23,  9, 16, 11,  3, 15, 16])


In [21]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_first 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers, output_dim):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size = input_dim,
                           hidden_size = hidden_dim,
                           num_layers = layers,
                           batch_first = True)
    # batch_first = True : 필요하다. x input 형태 (batch_size, seq_length, features)로 인식하기 위함.
    self.fc = torch.nn.Linear(hidden_dim, output_dim)
    # output_dim 없이 어떻게 선형 레이어 완성?
    # output이 글자를 예측하는 것, output_dim이 같은 input_dim
    # Task 2에 생략
  
  def forward(self, x):
    # RNN 레이어를 통해 x 처리
    # 여기서 x의 차원은 (batch_size, seq_length, input_dim)
    x, hidden = self.rnn(x)
    
    # Reshape output for fc layer
    # fc layer는 2D input 받기 때문에
    batch_size, sequence_length, hidden_dim = x.shape
    
    x = x.contiguous().view(batch_size * sequence_length, hidden_dim)
    # contiguous() : batch_size * seq_length를 유지하면서 나머지 차원을 평탄화
    
    # 선형 레이어 통과
    x = self.fc(x)
    
    # Reshape back to (batch size, sequence length, output dimension)
    x = x.view(batch_size, sequence_length, -1)
    
    return x

In [22]:
# net = Net(vocab_size, hidden_size, 2)

net = Net(vocab_size, hidden_size, 2, vocab_size)

In [23]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [24]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([183, 15, 26])


In [25]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

ss''''s's'ssss's'sss's'ssssss's'ss''s's'ss''''''s''''''s''s'ss'''ssss'ss's's'ssss's'''s'ss's's's's'ss'ss''''ss''sss'ss'ss''s'ss''sss'ss'''ssss'ss's's'ssss's'sss's's's's'ss'sss''ss's'ss's'ss'''s's's
a' '' ''   a'' ''t'' ''t''''''''tta't' '  t'' '' t ''t t'  '' ' t'''' '' ''   a'' 't ''t'' ''t ' t'a'tatt'' '  ''' '''' '' ' ''''  't' ' t'''' '' ''   a'' ''t'' ''t tt' '''a't '''' ' 't'''  ' t t  
a          t                                                                  t                                                                        t                                             
a                                                                                                                                                                                                    
                                                                                                                                                                                                     
          

rick walls are there tor arteason anl yor base wor then  thet the trick walls are kt there th  a t ts ou e tat tether tn thin wal thet the trick walls are there th  aor ts oow talll ae ta t thenk s
rick walls are there tor arteason anl yot base wor then  thet the trick walls are kt there th  a t us ou e tat tether tn thin wal thet the brick walls are there th saor us oow talll ae ta t thenk s
rick walls are there tor arteason anl yot base wor then  thit the brick walls are kt there th se t us ou e tat tetheretn thin wal thet the brick walls are there th soor us oow talll we ta t thenk s
rick walls are there tor arteason anl yot buse wor then  thit the brick walls are kt there th se t us oute tat tether tn thin wal thet the brick walls are there th soor us oow tadll we ta t thenkns
rick walls are there tor arreason anl yow buse wor then  that the brick walls are kt there th  ert us oute tat tether tn thin wal thet the brick walls are there th soow us oow tadll we ta t thenkns
rick walls

In [26]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there to show us how badly we want thinks."

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

In [29]:
#1. 생성할 문장 데이터

sentence = ("아직 정산요청 금액을 송금하지 않으셨나요? "
            "금액 확인 후 송금해주세요. "
            "정산요청 메시지나 정산현황을 통해 송금하지 않은 경우 알림 메시지가 발송될 수 있어요.")

#2. 문자 집합 만들기
world_set = list(set(sentence))

vocab = {char: i for i, char in enumerate(world_set)}

#3. 문자 집합 크기 확인

vocab_size = len(vocab)

#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10
learning_rate = 0.01

#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i + sequence_length]
  y_str = sentence[i+1 : i+1 + sequence_length]
#   print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])
    
##6. 입력 시퀀스에 대해 원핫인코딩 수행

x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_first 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size = input_dim,
                           hidden_size = hidden_dim,
                           num_layers = layers,
                           batch_first = True)
    self.fc = torch.nn.Linear(hidden_dim, input_dim)
  
  def forward(self, x):
    # RNN 레이어를 통해 x 처리
    # 여기서 x의 차원은 (batch_size, seq_length, input_dim)
    x, hidden = self.rnn(x)
    
    # Reshape output for fc layer
    # fc layer는 2D input 받기 때문에
    batch_size, sequence_length, hidden_dim = x.shape
    x = x.contiguous().view(batch_size * sequence_length, hidden_dim)
    # contiguous() : batch_size * seq_length를 유지하면서 나머지 차원을 평탄화
    
    # 선형 레이어 통과
    x = self.fc(x)
    
    # Reshape back to (batch size, sequence length, output dimension)
    x = x.view(batch_size, sequence_length, -1)
    return x

net = Net(vocab_size, hidden_size, 2)

##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

##14. 출력 크기 점검
outputs = net(X)

##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

predict_str

'직 정산요청 금액을 송금하지 않으셨나요? 금액 확인 후 송금해주세요. 정산요청 메시지나 정산현황을 통해 송금하지 않은 경우 알림 메시지가 발송될 수 있어요.'

In [30]:
world_set

['은',
 '수',
 '금',
 '해',
 '주',
 '경',
 '하',
 '현',
 '후',
 '.',
 '정',
 '으',
 '발',
 '셨',
 '어',
 '있',
 '직',
 '산',
 '세',
 '우',
 ' ',
 '청',
 '인',
 '황',
 '될',
 '시',
 '확',
 '않',
 '통',
 '아',
 '가',
 '?',
 '액',
 '알',
 '요',
 '송',
 '나',
 '지',
 '메',
 '림',
 '을']